# Atmo Model Training Notebook

Train an Atmo Model using `usl_models` lib.

In [ ]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import keras
from datetime import datetime, timedelta
import os, time
from usl_models.atmo_ml.model import AtmoModel, AtmoModelParams
from usl_models.atmo_ml import dataset
from google.cloud import storage

import logging

logging.getLogger().setLevel(logging.INFO)

# climateiq-study-area-feature-chunks/NYC_Heat/NYC_summer_2000_01p
# Define bucket names and folder paths
data_bucket_name = "climateiq-study-area-feature-chunks"
label_bucket_name = "climateiq-study-area-label-chunks"
time_steps_per_day = 6
batch_size = 4

sim_dirs = [
    ('NYC_Heat_Test', [
        'NYC_summer_2000_01p',
        # 'NYC_summer_2010_99p',
        # 'NYC_summer_2015_50p',
        # 'NYC_summer_2017_25p',
        # 'NYC_summer_2018_75p'
    ]),
    ('PHX_Heat_Test', [
        # 'PHX_summer_2008_25p',
        # 'PHX_summer_2009_50p',
        # 'PHX_summer_2011_99p',
        # 'PHX_summer_2015_75p',
        # 'PHX_summer_2020_01p'
    ])
]

sim_names = []
for sim_dir, subdirs in sim_dirs:
    for subdir in subdirs:
        sim_names.append(sim_dir + '/' + subdir)

print(sim_names)
client = storage.Client(project="climateiq")
feature_bucket = client.bucket(data_bucket_name)
label_bucket = client.bucket(label_bucket_name)


In [ ]:
inputs, label = dataset.load_day(
    date=datetime.strptime("2000-05-25", dataset.DATE_FORMAT),
    sim_name=sim_names[0],
    feature_bucket=feature_bucket,
    label_bucket=label_bucket,
)
label.shape

In [ ]:
train_frac = 0.8

# Create training dataset with fused spatiotemporal data
train_ds = dataset.load_dataset(
    data_bucket_name=data_bucket_name,
    label_bucket_name=label_bucket_name,
    sim_names=sim_names,
    dates=["2000-05-25"],
).batch(batch_size=1)
for inputs, labels in train_ds:
    print(labels.shape)
    break

In [ ]:
# Create training dataset with fused spatiotemporal data
train_ds = dataset.load_dataset(
    data_bucket_name=data_bucket_name,
    label_bucket_name=label_bucket_name,
    sim_names=sim_names,
    dates=["2000-05-25"],
).batch(batch_size=batch_size)

# Create validation dataset with fused spatiotemporal data
val_ds = dataset.load_dataset(
    data_bucket_name=data_bucket_name,
    label_bucket_name=label_bucket_name,
    sim_names=sim_names,
    dates=["2000-05-25"],
).batch(batch_size=batch_size)

In [ ]:
# Initialize the Atmo Model
model_params = AtmoModelParams()
model = AtmoModel(model_params)

In [ ]:
# Create a unique log directory by appending the current timestamp
log_dir = os.path.join("./logs", "run_" + time.strftime("%Y%m%d-%H%M%S"))

# Set up TensorBoard callback
tb_callback = keras.callbacks.TensorBoard(log_dir=log_dir)
print(log_dir)

In [ ]:
# Train the model
model.fit(train_ds, val_ds, epochs=150, callbacks=[tb_callback])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Get predictions from the validation set
predictions = model._model.predict(val_ds)  # Use the underlying Keras model

# Assuming the structure of val_ds returns (input_data, ground_truth)
for input_data, ground_truth in val_ds.take(1):  # Taking just one batch from val_ds
    # Get predicted labels
    predicted_labels = model._model.predict(input_data)
    
    # Compute shared vmin and vmax for consistent color range
    vmin = min(
        np.min(ground_truth[:, 0, :, :, 0]), 
        np.min(predicted_labels[:, 0, :, :, 0])
    )
    vmax = max(
        np.max(ground_truth[:, 0, :, :, 0]), 
        np.max(predicted_labels[:, 0, :, :, 0])
    )
    
    # Visualize the first sample
    fig, axes = plt.subplots(1, 2, figsize=(12, 6), dpi=150)  # Higher DPI for quality
    
    # Ground Truth Visualization
    img1 = axes[0].imshow(
        ground_truth[0, 0, :, :, 0], 
        cmap='viridis', 
        vmin=90, 
        vmax=100
    )
    axes[0].set_title('Ground Truth')
    plt.colorbar(img1, ax=axes[0], fraction=0.046, pad=0.04)

    # Prediction Visualization
    img2 = axes[1].imshow(
        predicted_labels[0, 0, :, :, 0], 
        cmap='viridis', 
        vmin=vmin, 
        vmax=vmax
    )
    axes[1].set_title('Predicted Labels')
    plt.colorbar(img2, ax=axes[1], fraction=0.046, pad=0.04)

    plt.tight_layout()
    plt.show()
    break  # Break after visualizing one batch

In [ ]:
inputs, labels = next(iter(train_ds))
{key: tensor.shape for key, tensor in inputs.items()}

In [ ]:
model._model.summary()

In [ ]:
# Test calling the model on some training data
inputs, labels = next(iter(train_ds))
prediction = model.call(inputs)
print("Prediction shape:", prediction.shape)